In [1]:
import glob
import numpy as np
import pandas as pd

In [2]:
def get_pos_freq_dict(fn):
    f = open(fn, 'r')
    pos_freq_dict = {}
    for line in f:
        linfo = line.strip().split("\t")
        pos_freq_dict[linfo[0]] = float(linfo[1])
    return pos_freq_dict

def compile_tables(INPUT_DIR, OUTPUT_FILE, THRESHOLD=0.25):
    
    exp_cond_data = {}
    column_headers = []
    files = sorted(glob.glob(INPUT_DIR + "/*"))
    
    for fn in files:
        if("remote" in fn.split("/")[-1]):continue
        exp_name = fn.split("/")[-1].split(".")[0]
        column_headers.append(exp_name)
        pos_freq_dict = get_pos_freq_dict(fn)
        exp_cond_data[exp_name] =  pos_freq_dict

    ### Union keys
    unioned_keys = set()
    for exp_name in exp_cond_data:
        pos_freq_dict = exp_cond_data[exp_name]
        unioned_keys |= set(pos_freq_dict.keys())

    ### Filter and write out data
    f = open(OUTPUT_FILE, 'w')
    f.write("LIGxLIG:GPCRDB1:GPCRDB2\t" + "\t".join(column_headers) + "\n")
    row_string_tup = []
    for key in unioned_keys:
        row_info = [key]
        for exp_name in column_headers:
            pos_freq_dict = exp_cond_data[exp_name]
            if(key in pos_freq_dict):
                row_info.append(pos_freq_dict[key])
            else:
                row_info.append(0)
        if(np.mean(row_info[1:]) > THRESHOLD):
            f.write("\t".join(map(str,row_info)) + "\n")


In [3]:
INACTIVE_RECEPTORS="/scratch/PI/rondror/akma327/DynamicNetworks/data/crystal-analysis/ligand-wetness/remote_ligand_to_receptor_contacts/021617/inactive_receptors"
OUTPUT_FILE="/scratch/PI/rondror/akma327/DynamicNetworks/data/crystal-analysis/ligand-wetness/remote_ligand_to_receptor_contacts/021617/inactive_receptors/remote_ligand_to_receptor_contacts_inactive_receptors.txt"
compile_tables(INACTIVE_RECEPTORS, OUTPUT_FILE)

In [4]:
ACTIVE_RECEPTORS="/scratch/PI/rondror/akma327/DynamicNetworks/data/crystal-analysis/ligand-wetness/remote_ligand_to_receptor_contacts/021617/active_receptors"
OUTPUT_FILE="/scratch/PI/rondror/akma327/DynamicNetworks/data/crystal-analysis/ligand-wetness/remote_ligand_to_receptor_contacts/021617/active_receptors/remote_ligand_to_receptor_contacts_active_receptors.txt"
compile_tables(ACTIVE_RECEPTORS, OUTPUT_FILE)